# Face Recognition System

# Importing Libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

# Pre Processing

In [2]:
# Training set

train_datagen = ImageDataGenerator(
    rescale=1./255, #for standardising
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
    'dataset/training_set', #dataset folder path
    target_size=(64,64), #target size (pixel, pixel)
    batch_size=32,
    class_mode = 'categorical' #binary or categorical output
)

# Test set

test_datagen = ImageDataGenerator(
    rescale=1./255 #no other parameter cause it be test set but still need to festure scale
)

test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

Found 116 images belonging to 3 classes.
Found 23 images belonging to 3 classes.


In [3]:
test_set.class_indices #to check which class is assigned to which index

{'Anit': 0, 'Arhat': 1, 'Vivek': 2}

In [4]:
Mapping = {}
for faceValue,faceName in zip(test_set.class_indices.values(),test_set.class_indices.keys()):
    Mapping[faceValue]=faceName

print(Mapping)

{0: 'Anit', 1: 'Arhat', 2: 'Vivek'}


In [5]:
OutputNeurons = len(Mapping)
print(OutputNeurons)

3


# Building CNN

In [6]:
# Initialising the CNN

cnn = tf.keras.models.Sequential()

#Convolution and max Pooling layer 1

cnn.add(tf.keras.layers.Conv2D(filters=32, #number of filters
                               kernel_size=3, #size of each filter
                               activation='relu', #activation function
                               input_shape=[64,64,3])) #64 by 64 is the size & For coloured images, for B/W [64,64,1]

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, #to set the size of the window used for pool
                                  strides=2)) #to set the number of pixels by which the window shifts per iteration

#Convolution and max Pooling layer 2

cnn.add(tf.keras.layers.Conv2D(filters=32, #number of filters
                               kernel_size=3, #size of each filter
                               activation='relu', #activation function
                               )) #input shape is not required for any layer other than first

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, #to set the size of the window used for pool
                                  strides=2)) #to set the number of pixels by which the window shifts per iteration

#Flattening

cnn.add(tf.keras.layers.Flatten())

#Full Connection

cnn.add(tf.keras.layers.Dense(units=128, activation='relu')) #A higher number of neurons for more complex datas

#Output Layer

cnn.add(tf.keras.layers.Dense(units=OutputNeurons, activation='softmax')) #For non-binary classification

# Training CNN

In [7]:
# Compiling CNN

cnn.compile(optimizer = 'adam', #for binary
            loss = 'categorical_crossentropy',
            metrics = ["accuracy"])

# Training the CNN on the Training set and evaluating it on the Test set

cnn.fit(x = training_set, #Training set value
        validation_data=test_set, #Test set value
        epochs = 25) #No. of epochs

Epoch 1/25
4/4 [==============================] - 1s 155ms/step - loss: 0.9856 - accuracy: 0.5259 - val_loss: 1.0013 - val_accuracy: 0.4348
Epoch 2/25
4/4 [==============================] - 0s 97ms/step - loss: 0.8287 - accuracy: 0.6810 - val_loss: 0.8001 - val_accuracy: 0.8696
Epoch 3/25
4/4 [==============================] - 0s 97ms/step - loss: 0.7430 - accuracy: 0.7500 - val_loss: 0.6624 - val_accuracy: 0.8261
Epoch 4/25
4/4 [==============================] - 0s 105ms/step - loss: 0.5951 - accuracy: 0.8190 - val_loss: 0.6121 - val_accuracy: 0.8261
Epoch 5/25
4/4 [==============================] - 0s 101ms/step - loss: 0.4828 - accuracy: 0.8276 - val_loss: 0.4975 - val_accuracy: 0.8261
Epoch 6/25
4/4 [==============================] - 0s 95ms/step - loss: 0.4395 - accuracy: 0.8534 - val_loss: 0.4755 - val_accuracy: 0.8261
Epoch 7/25
4/4 [==============================] - 0s 99ms/step - loss: 0.3430 - accuracy: 0.8707 - val_loss: 0.4319 - val_accuracy: 0.8261
Epoch 8/25
4/4 [========

# Making Simple Prediction

In [8]:
import numpy as np
from tensorflow.keras.preprocessing import image
import clickNsave
import sys

img_path =  clickNsave.ClicknSave() # path of single input, '-1' if no faces detected, '0' if more than 1 face detected

if img_path == '-1':
    print('No faces detected')
    sys.exit()

elif img_path == '0':
    print('More than 1 face detected')
    sys.exit()

test_image = image.load_img(img_path, # path of single input
                            target_size = (64,64)) # resize to get size as trained
test_image = image.img_to_array(test_image) # to convert it into a numpy array
test_image = np.expand_dims(test_image, # to add extra dimension (for the batch number)
                            axis = 0) # to put the batch number dimension as the first one

result = cnn.predict(test_image)
print('Prediction is: ', Mapping[np.argmax(result)])

1 face(s) cropped and saved in 'C:\Drive D\ArhatPersonal\ML\Practice\Proj_FaceRecog\dataset\cropped_faces' folder.
1/1 [==============================] - 0s 59ms/step
Prediction is:  Anit


# To add to dataset

In [9]:
import shutil

Q = input('Did we get it right? (y/n): ')
if Q == 'y':
    shutil.move(img_path, 'dataset/training_set/'+Mapping[np.argmax(result)])

    import os

    #Counting number of files in the destination folder

    # folder path
    dir_path = 'dataset/training_set/'+Mapping[np.argmax(result)]
    count = 0
    # Iterate directory
    for path in os.listdir(dir_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path, path)):
            count += 1
    print('File count:', count)

    #Renaming the file to the next number

    os.rename(dir_path+'/'+'face_0.jpg', dir_path+'/Added'+str(count)+'.png')

    

elif Q == 'n':
    print('Sorry for the inconvenience. We will try to improve our model.')

else:
    print('Invalid input.')

Sorry for the inconvenience. We will try to improve our model.
